# Analysis of Treasury Bill Pricing and Interest Rate Risk

## Background
T-Bills, also known as Treasury bills, are financial instruments with short-term due dates ranging from a few days to 52 weeks. These fixed-income investments have zero coupons, which means that no coupon payments are made during the term. Instead, the price of T-Bills is calculated so that the receiver gets the face (par) value ($V_{P}$) at the end of the term. T-Bills are available for specific periods of 4, 8, 13, 26, and 52 weeks. The fair price ($V_{B}$) of a T-Bill is the future face value ($V_{P}$) that is discounted to today’s value by the effective market interest rate ($\bar{r}$), which is assumed to be constant over the lifetime of the T-Bill.

<div>
    <center>
        <img src="figs/Fig-ZC-Schematic.png" width="440"/>
    </center>
</div>

### Pricing
A zero-coupon T-bill with an annual effective market interest rate $\bar{r}$, which is specified at the time of purchase, and a term of T years has a _fair price_ of:

$$
V_{B} = \mathcal{D}_{T,0}^{-1}\cdot{V_{P}}
$$

where $\mathcal{D}_{T,0}$ denotes the discount factor from the time of the auction (0) to the term of T-bill, i.e., $0\rightarrow{T}$. Typically, a constant effective annualized rate of interest $\bar{r}$ (and discrete compounding) is used to compute the discount factor $\mathcal{D}_{T,0} = (1+\bar{r})^{T}$. However, continuous compounding or period-specific discount factors of the form:

$$
\mathcal{D}_{T,0} = \left[\prod_{j=0}^{T-1}\left(1+r_{j+1,j}\right)\right]
$$

can also be used. The $r_{j+1,j}$ values, called short rates, describe the interest rate earned between period $j\rightarrow{j+1}$.

## Project
In this project, you purchased a 52-week T-bill with a par value $V_{P}=100.0$ USD and an effective market rate of interest $\bar{r}$ (annualized) trading day $i$ of `2022`. However, on trading day $j$ of `2022`, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` it to term?

### Objectives
 1. Your first objective is to load historical T-bill interest rate data sets from `2019` to `2022` and store these in a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries). 
 1. Your second objective is to develop a strategy to identify trade signals for your T-bill. You will test your strategy as you move forward and backward between January 03, 2022, and December 31, 2022.

## Setup
In the following code block we setup the computational aspects of the problem by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this excercise and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.
* The `MyBinaryLatticeNodeModel` and `MySymmetricBinaryLatticeModel` mutable types hold information about the nodes, and the binomial lattice, respectively, for the short-rate calculations. 

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

##### `loadratesfile(year::String = "2022") -> DataFrame`

> This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.

##### `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel`

> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

##### `build(model::Type{MySymmetricBinaryLatticeModel}, data::NamedTuple)::MySymmetricBinaryLatticeModel` and `build(model::Type{MyBinaryLatticeNodeModel}, data::NamedTuple)::MyBinaryLatticeNodeModel` 

> These factory functions take information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument and construct a node or the binomial short-rate lattice model, respectively. 

##### `price(model::MyUSTreasuryBillModel, compounding::T) -> MyUSTreasuryBillModel where T <: AbstractCompoundingModel`

> This function takes a `MyUSTreasuryBillModel` instance, and a compounding model and returns an updated `MyUSTreasuryBillModel` instance that holds the price $V_{B}$ value in the `price` field of the `MyUSTreasuryBillModel` instance.

##### `𝔼(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64` and `𝕍(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64`

> These functions take a`MySymmetricBinaryLatticeModel` instance, and the level and the tree (`0`-based) and compute the [expectation](https://en.wikipedia.org/wiki/Expected_value) and [variance](https://en.wikipedia.org/wiki/Variance) of the short-rates for the given level (time-step) of the lattice. 

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


## Set constant values 
We will need several constants, regardless of the year we choose to analyze. First, we will always analyze 52-week data, so we encode this choice in the `key` variable. Next, we'll assume a discrete compounding model; thus, we instantiate an instance of the `DiscreteCompoundingModel` type and then store this instance in the `compounding` variable. Lastly, the T-bill’s face (par) value will always be $V_{P}$ = 100.0 USD.

In [2]:
number_of_weeks_per_year = 52;
key = Symbol("52 WEEKS COUPON EQUIVALENT");
Vₚ = 100.0;
T = (52.0/number_of_weeks_per_year);
compounding = DiscreteCompoundingModel();

## Objective 1: Load historical interest rate dataset
To begin, we gather interest rate data from `2019` to `2022`, using rates from recently auctioned bills collected by the Federal Reserve Bank of New York on every business day.  For this analysis, we use the `52 WEEKS COUPON EQUIVALENT` data as a proxy for the effective market interest rate $\bar{r}$ on each trading day for a 52-week T-bill.

### Implementation
We store the rate information in a `rate_data_dictionary` variable, which is a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) with keys representing years (from the `years` array, type `String`) and values as a `DataFrame` holding rate values. We load each year by iterating through the `years` array and passing the corresponding `year` parameter to the `loadratesfile(...)` function:

In [3]:
rate_data_dictionary = Dict{String,DataFrame}();
years = ["2019","2020","2021","2022"];
for year ∈ years
    
    ### BEGIN SOLUTION
    rate_data_dictionary[year] = loadratesfile(year=year);
    ### END SOLUTION
end

Now that have loaded the rates dataset, let's analyze `2022`, and save our selection in the variable `selected_year`. Then access the corresponding rate values `DataFrame` for the `selected_year` from the `rate_data_dictionary`. Store the rate data in the `dataset` variable:

In [4]:
### BEGIN SOLUTION
selected_year = "2022"
dataset = rate_data_dictionary[selected_year];
### END SOLUTION

## Objective 2: Identify possible treasury bill trade signals
Deciding whether to `buy`, `sell`, or `hold` a T-bill, or any other security, depends on your investment objectives. Treasury bills are generally considered secure, short-term investments, but their prices can fluctuate depending on the interest rate. If the price of a T-bill changes, you could `sell` it before maturity and earn an immediate capital gain or loss. However, it's crucial to know when this makes sense.

To start answering this question, let's examine our historical data set to identify potential trade signals, which indicate changes in the T-bill's price that could result in a capital gain or loss. We'll use the [log return](https://en.wikipedia.org/wiki/Rate_of_return) $R_{ij}$ between two dates ($i\geq{j}$) to measure the potential gain or loss:

$$
R_{i,j} = 100\cdot{\ln\left(\frac{V_{B,i}}{V_{B,j}}\right)}
$$

Here, $V_{B,k}$ refers to the price of a 52-week T-bill on day $k$ in the dataset, and $\ln$ denotes the [natural logarithm](https://en.wikipedia.org/wiki/Natural_logarithm). If you have two possible actions $a\in\left\{\text{sell},\text{hold}\right\}$ you can propose the following policy: Implement a `sell` action if $R_{ij}>\alpha$, where $\alpha>0$ is a user-specified trade threshold. Otherwise, implement a `hold` action.

### Prerequisite: Compute the price of a 52-week T-bill for each trading day of 2022
To calculate the cost of a 52-week T-bill for each trading day in 2022, start by finding out how many trading days are in `2022` using the `nrow(…)` function. Save this number in the `number_of_trading_days` variable. Then create a one-dimensional `models` array containing instances of `MyUSTreasuryBillModel`. Finally, using a `for` loop, construct `MyUSTreasuryBillModel` instances,  and compute the price, for a 52-week treasury bill using the effective market rate of interest $\bar{r}$ (found in the `dataset` variable) for each trading day. Store these instances in the `models` array.

__Reminder__: the `dataset` is organized from the newest (Dec-31-2022) to oldest (Jan-03-2022).

In [5]:
number_of_trading_days = nrow(dataset)
models = Array{MyUSTreasuryBillModel,1}(undef, number_of_trading_days);
for i ∈ 1:number_of_trading_days
    
    ### BEGIN SOLUTION
    r̄ = dataset[i,key]*(1/100)
    model = build(MyUSTreasuryBillModel, (
        par = Vₚ, T = T, rate=r̄)) |> compounding
    
    models[i] = model;
    ### END SOLUTION
end

### Compute the log return $R_{ij}$ array for all combinations of $(i,j)$:
Compute the array of log returns for the 52-week T-bills in 2022. First, define a two-dimensional array that holds the log return values (type `Float64`) named `log_return_array`. The `log_return_array` will be a `number_of_trading_days` $\times$ `number_of_trading_days` matrix where the $(i,j)$ value is $R_{ij}$. Initialize the elements of `log_return_array` with zeros using the [fill!](https://docs.julialang.org/en/v1/base/arrays/#Base.fill!https://docs.julialang.org/en/v1/base/arrays/#Base.fill!) function:

In [6]:
log_return_array = Array{Float64,2}(undef, number_of_trading_days, number_of_trading_days)
fill!(log_return_array,0.0);

Next, calculate the $R_{i,j}$ values $\forall{i,j}$. First, reverse the order of the `models` array (so that it runs from the oldest to newest dates) using the [reverse](https://docs.julialang.org/en/v1/base/arrays/#Base.reverse-Tuple{AbstractVector}) function. Store the reversed array in the `reversed_models_array` variable. Then calculate the elements of the `log_return_array` using nested `for` loops, each running between `1...number_of_trading_days`. The outer `for` loop will iterate over the rows `log_return_array` while the inner loop will iterate over the columns of `log_return_array`:

In [7]:
reversed_models_array = reverse(models);
for i ∈ 1:number_of_trading_days
    
    purchased_model = reversed_models_array[i];
    purchase_price = purchased_model.price
    
    for j ∈ 1:number_of_trading_days
        next_model = reversed_models_array[j];
        next_price = next_model.price
        
        ### BEGIN SOLUTION
        log_return_array[i,j] = log(next_price/purchase_price)*100.0;
        ### END SOLUTION
    end
end

In this formulation of the `log_return_array` the rows correspond to the index of the date the T-bill was purchased, while the columns denote the days the T-bill could be sold. Thus, values of $i,j$ such that $i<j$ denote forward time, while $i>j$ denote reverse time (we have a magical time machine that allows us to sell the T-bill beofre we bought it). 

### We purchased the T-bill at $i=1$, are there any sell signals in 2022?
A `sell` signal will have (at a minimum) a value of $\alpha>0$. Are there any dates where this is true in `2022`? We can implement the search for sell signals using a `for` loop in which we iterate through the columns of the first row of `log_return_array`. Alternatively, we can use the [findall](https://docs.julialang.org/en/v1/base/arrays/#Base.findall-Tuple{Function,%20Any}) function in Julia:

In [8]:
sell_index_vector = findall(x->x>0, log_return_array[1,:]);
sell_date_array = reverse(dataset)[sell_index_vector,:Date]

1-element Vector{String15}:
 "01/04/2022"

### We purchased the T-bill on the last day of the year, and used a time-machine, are there any sell signals in 2022?
If we move backward through the columns on the last row of `log_return_array`, we are moving from the last trading day (Dec-31) to the first (Jan-03). Like the previous search, we can implement the search for sell signals using a `for` loop in which we iterate through the columns of the first row of `log_return_array`. Alternatively, we can use the [findall](https://docs.julialang.org/en/v1/base/arrays/#Base.findall-Tuple{Function,%20Any}) function in Julia:

In [9]:
### BEGIN SOLUTION
sell_index_vector = findall(x->x>0, log_return_array[end,:]);
sell_date_array = reverse(dataset)[sell_index_vector,:Date]
### END SOLUTION

229-element Vector{String15}:
 "01/03/2022"
 "01/04/2022"
 "01/05/2022"
 "01/06/2022"
 "01/07/2022"
 "01/10/2022"
 "01/11/2022"
 "01/12/2022"
 "01/13/2022"
 "01/14/2022"
 "01/18/2022"
 "01/19/2022"
 "01/20/2022"
 ⋮
 "12/09/2022"
 "12/13/2022"
 "12/14/2022"
 "12/15/2022"
 "12/16/2022"
 "12/19/2022"
 "12/20/2022"
 "12/21/2022"
 "12/22/2022"
 "12/23/2022"
 "12/28/2022"
 "12/29/2022"

In [13]:
# Test 1: length(rate_data_dictionary) = 4
@assert length(rate_data_dictionary) == 4

# Test 2: dataset should not be nothing, and should type DataFrame
@assert (isnothing(dataset) == false && typeof(dataset) == DataFrame)

# Test 3: models should have length = number_of_trading_days
@assert length(models) == number_of_trading_days

# Test 4: log_return_array should have non-zero values
index_list_nozero = findall(x->x!=0.0, log_return_array);
@assert isempty(index_list_nozero) == false

# Test 5: sell_index_vector should have 229 elements -
@assert length(sell_index_vector) == 229